# Preamble

## Are You in the Right Place?

The following is part of a multi-part introduction to data science for those in the legal profession. The full collection of materials can be found at [https://www.codingthelaw.org](https://www.codingthelaw.org). This if from 
[Level 7](https://www.codingthelaw.org/level/7/). 

# How to Train a Model

Warning! We are approaching the Danger Zone. That is, you're about to learn just enough to be dangerous. With great power comes great responsibility. 

![Data Science](http://static1.squarespace.com/static/5150aec6e4b0e340ec52710a/t/51525c33e4b0b3e0d10f77ab/1364352052403/Data_Science_VD.png?format=750w) 

Alright, let's go ahead and load Pandas. You may remeber this library from the data wrangeling notebook. 

In [ ]:
import pandas as pd

Now we can load the data that we cleaned in as part of [Level 6](https://www.codingthelaw.org/level/6/). 

In [ ]:
processed_table = pd.read_csv('processed_table_only_numbers.csv') 
processed_table.head()

## Linear Regression

I'm now going to make a set of tables to be used in training and testing some models. This process should sound familiar from our prior discussions of training and testing. 

Remember, linear regression is used to predict continuous values (things that fall on a number line). I know we talked earlier (in the last notebook) about predicting snow days, but that's not a very good use case for linear regression. It's designed for predicting some number on a number line. So here I'm going to try to predict today's high using yesterday's. So I'll need the target (max) and the feature (yesterdays_high).

In [ ]:
lin_df = processed_table[[
                               'max',
                               'yesterdays_high'
                               ]].copy()
lin_df.head()

In practice, we'd probably want to use more than one feature, but this one-to-one construction works well for an introduction. I susspect we'll see that yesterday's high is corelated with today's. That is, as one goes up, the other does the same. Let's see.

But first, what about overfitting? Yep, I'm using a vocabulary word from the [reading](https://web.archive.org/web/20181204182136/https://lawyerist.com/big-bias-big-data/). To address that concern let's do a proper training of the data using training and holdout dataframes. That is, we'll set some amount of the data aside (holdout), train our model on what remains, and then check and see how the model does on the data it hasn't seen. 

In [ ]:
print("size of all data",len(lin_df))

In [ ]:
# create a dataframe containing a random sample of lin_df rows
lin_holdout = lin_df.sample(frac=0.20) # 0.20 = 20%

# create a dataframe that conatins the rows from lin_df except for those in lin_holdout
lin_training = lin_df.loc[~lin_df.index.isin(lin_holdout.index)]

print("size of training",len(lin_training))

Okay, let's load some librarys and perform a linear regression.

In [ ]:
import matplotlib
matplotlib.use("agg")
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.graphics.regressionplots import abline_plot

In [ ]:
from statsmodels.formula.api import ols

model = ols("max ~ yesterdays_high", lin_training).fit()

# scatter-plot data
ax = lin_training.plot(x='yesterdays_high', y='max', kind='scatter')

# plot regression line
fig = abline_plot(model_results=model, ax=ax)

In [ ]:
# I like the statsmodels regression library because it comes with a nice summary.
# So let's load the library and run the regression on our training data. 
# There are two main numbers we should focus on below, the R-squared and the P-values. 

model.summary()

We have yet to check the model against our holdout data, and it so happens that another implementation of the linear regression algo can be found over at scikitlearn. It makes evaluating against the holdout data rather simple. 

In [ ]:
# Rerun with SciKitLearn because it's easy to check accuracy

from sklearn import linear_model
from sklearn import metrics

# make a training dataframe containing just features
features_train = lin_training.drop("max", axis=1).values

# make a training dataframe containing only the target
target_train = lin_training["max"].values

# make a testing/holdout dataframe containing just features
features_test = lin_holdout.drop("max", axis=1).values

# make a testing/holdout dataframe containing only the target
target_test = lin_holdout["max"].values

# run the regression
lm = linear_model.LinearRegression()
reg = lm.fit(features_train, target_train)
print("R squared (training):",lm.score(features_train,target_train)) # this sould be the same as above

# create a dataframe of predictions based on the test features
pred = reg.predict(features_test)
# compare these predictions to the actual values
variance = metrics.r2_score(target_test, pred)
print("Variance score (test):",variance)

So our model "explains" about 70% of the variance for both our training and holdout data. It turns out the temperature yesterday is a pretty good predictor for the temperature today. Imagine how good we'd be at prediction if we added a few other relevant features or if we lived in California (I'm told the weather is always the same ;)

## Classifiers

Now let's get back to predicting snow days. To predict a snow day we need a model that spits out a categorical value (i.e., a label like yes or no). This differs from a continuous value, like those we were looking at above. To make things simple, we're going to have yes = 1 and no = 0. That being said, let's clean up our processed table accordingly.

In [ ]:
# Note: the question of what features to include is beyond the scope of this post. 
# We'll just say these all look like they could help predict a snow day.

class_df = processed_table[[
                               'accum', 
                               'min', 
                               'max',
                               'wind',
                               #'yesterdays_high',
                               'closed'
                               ]].copy()
class_df.head()

Here's a little function to help evaluate how good our classifiers are. It produces the terms for a confusion matrix. See https://en.wikipedia.org/wiki/Confusion_matrix

In [ ]:
# I'll need these libraries to make it work
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.metrics import roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
    
def evaluate(prob, pred, labels_test, verbose=1):
    acc = accuracy_score(labels_test,pred)
    ones = sum(labels_test)/len(labels_test)
    zeros = (1 - ones)
    tn, fp, fn, tp = confusion_matrix(labels_test, pred).ravel()

    recall_ = tp / (tp + fn) 
    precision_ = tp / (tp + fp)
    f1 = (2 / ((1/recall_)+(1/precision_)))

    false_positive_rate, true_positive_rate, thresholds = roc_curve(labels_test,prob)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    
    if verbose==1:
        
        plt.rcParams['figure.figsize'] = [14, 4]
        
        plt.subplot(1, 3, 1)
        plt.title('Receiver Operating Characteristic')
        plt.plot(false_positive_rate, true_positive_rate, 'b',
        label='AUC = %0.2f'% roc_auc)
        plt.legend(loc='lower right')
        plt.plot([0,1],[0,1],'r--')
        plt.xlim([-0.1,1.2])
        plt.ylim([-0.1,1.2])
        plt.ylabel('True Positive (Hit) Rate')
        plt.xlabel('False Positive (False Alarm) Rate')

        precision, recall, thresholds = precision_recall_curve(labels_test,prob)

        plt.subplot(1, 3, 2)
        plt.title('Precision-Recall Curve')
        plt.plot(recall, precision,'b',
        label='AP = %0.2f'% precision.mean())
        plt.legend(loc='lower right')
        plt.xlim([-0.1,1.2])
        plt.ylim([-0.1,1.2])
        plt.ylabel('Recall')
        plt.xlabel('Precision')

        plt.subplot(1, 3, 3)
        plt.hist([labels_test,prob], label=["true","pred"], bins='auto')  # arguments are passed to np.histogram
        plt.title("Predictions vs Reality")
        plt.legend(loc='best')
        plt.xlim([-0.1,1.2])

        plt.tight_layout()
        plt.show()    

        print ("\n0s: %0.2f\tTrue Positives: %s\tAccuracy: %s "%(zeros,tp,acc))
        print ("1s: %0.2f\tTrue Negatives: %s\tAUC: %s"%(ones,tn,roc_auc))
        print ("\t\tFalse Positives: %s\tF1 Score: %s"%(fp,f1))
        print ("\t\tFalse Negatives: %s\tRecall (fract of actual yeas found): %s"%(fn,recall_))
        print ("\t\t\t\t\tPrecision (correctness of yeas predicted): %s\n"%precision_)        
        
        null = zeros
        if null < ones:
            null = ones
        
        if (acc>null) and (roc_auc>0.5) and (recall_>0.5) and (precision_>0.5):
            print("################")
            print("#     PASS     #")
            print("################")
            vpass = "Yes"
        else:
            print("################")
            print("#     FAIL     #")
            print("################")
            vpass = "No"
          
    #return ones,vpass,null,acc,roc_auc,recall_,precision_,f1

And of course we need training and holdout data.

In [ ]:
# create a dataframe containing a random sample of rows
class_holdout = class_df.sample(frac=0.20)

# create a dataframe that conatins the rows from except for those in holdout
class_training = class_df.loc[~class_df.index.isin(class_holdout.index)]

# make a training dataframe containing just features
features_train = class_training.drop("closed", axis=1).values

# make a training dataframe containing only the target
labels_train = class_training["closed"].values

# make a testing/holdout dataframe containing just features
features_test = class_holdout.drop("closed", axis=1).values

# make a testing/holdout dataframe containing only the target
labels_test = class_holdout["closed"].values

print("size of training",len(class_training))

# Show how many instances there are of closed (1) and opened (0)
print(class_training["closed"].value_counts())

Now just to keep ourselves honest, let's ask, "What percentage of the time is target 0?"

In [ ]:
print("Percentage of 0s: %s\n"%(len(class_df[class_df["closed"]==0])/len(class_df)))

# This is a useful question because it helps us not be overly impressed by the accuracy of our models. 
# How? Well, apparently, if I always guessed that we wouldn't have a snow day I'd be right ~98% of the time.

print(class_df["closed"].value_counts())

What follows is the feeding or our data to a bunch of different classification algorithms. The "best" models will have the highest F1 score. At least, that's how we'll define best at the moment. 

Note, they may or may not work depending on the contents of your training data. Since there are so few school closings, it's possible you won't get enough of them in the training data for the algos to work. When this happens you'll likely see a pink error message with something like "divide by zero encountered." If this happens and you really want the pink to go away, rerun the cell above where you create the training and holdout dataframes. When you use bigger datasets this shouldn't be an issue. 

I've provided links to documentation on each of the classification algos so you can investigate what they're doing under the hood. That being said, this documentation is probably not the best introduction. So you may want to consider Googling their names.

First up, 
## Logistic Regression 
http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(fit_intercept = False, C = 1e9)
clf_1 = model.fit(features_train, labels_train)
pred = clf_1.predict(features_test)
prob = clf_1.predict_proba(features_test)[:,1] 
print("Logistic Regression")
evaluate(prob, pred, labels_test)  

## Decision Tree Classifier
http://scikit-learn.org/stable/modules/tree.html#tree

In [ ]:
from sklearn import tree
clf_2 = tree.DecisionTreeClassifier(random_state=0,min_samples_split=2,max_leaf_nodes=2,min_samples_leaf=1)
clf_2 = clf_2.fit(features_train, labels_train)

print("Decsion Tree")

import graphviz 
dot_data = tree.export_graphviz(clf_2, out_file=None, 
                      feature_names=["accumulation" ,"min temp","max temp","wind"],  
                      class_names=["open","closed"],  
                      filled=True, rounded=True,  
                      special_characters=True)  
graph = graphviz.Source(dot_data)  
graph

In [ ]:
pred = clf_2.predict(features_test)
prob = clf_2.predict_proba(features_test)[:,1] 
print("Decision Tree")
evaluate(prob, pred, labels_test)

## Random Forest
http://scikit-learn.org/stable/modules/ensemble.html#forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_3 = RandomForestClassifier()
clf_3 = clf_3.fit(features_train, labels_train)
pred = clf_3.predict(features_test)
prob = clf_3.predict_proba(features_test)[:,1] 
print("Random Forest")
evaluate(prob,pred, labels_test)  

## Making Predictions 
To actually use your models we use `.predict()`. All you do is pass your model a list of feature values, and it will spit out its best guess re: answers. You can actually pass a multi-dimensional array of values (basically a table), but here we'll limit ourselves to a simple list. The ability to pass an array however, is why you see bracketed numbers after `.predict()`. They're asking that the output only focus on the bits we want. 

In [ ]:
# What classifier do you want to use? Above, each one is 
# given it's own number (e.g., clf_1, clf_2, etc.)
clf = clf_2

# Set the values for each of the above features: accum, min, max, wind
inputs = [30,-20,50,50]

guess = clf.predict([inputs])[0]
print("Model's Guess:",guess)

if guess == 0:
    probability = clf.predict_proba([inputs])[0][0]
else:
    probability = clf.predict_proba([inputs])[0][1]
    
print("Confidence the guess is right:",probability)

Okay, now back to that multi-dimensional array. `features_test` is just such an array. So `pred_[number]` is just a list of numbers (1= snow day and 0= no snow day) for that list. Here's the list of features (a list of lists with three numbers per "row"):  

In [ ]:
features_test[:3] # the [:3] is just a way to say, show only the first three "rows", kind of like .head()

You may have guessed it, but an array is just another datatype, it's kind of like a dataframe but it's a list of lists where the lists are numbers. Anywho. You can feed this array into your model like so:

In [ ]:
clf.predict(features_test)

And as you can see each of the numbers in `pred` corresponds to either a 1 (snow day) or a 0 (no snow day).

Let's say that for example, I wanted to take the data from above and see for what accumulations the algo predicted school would be closed. Then I could pass it our feature data (`features_test`), marry it to our acummulation data, and filter by `1`s. 

In [ ]:
features_test[:,0] # This is just the way to get the first 0ith column of data (accum) in the feature test array. 

So now I can produce a similar array of accumulations and marry the two together into a dataframe where I can filter out predictions of `0`. Like so:

In [ ]:
percentile_list = pd.DataFrame(
    {'accum': features_test[:,0], 'pred': clf.predict(features_test)
    })
percentile_list[percentile_list["pred"]==1]

ProTip: This only works because the order of the accumulation list and the prediction list are the same. If I wanted to do something like figure out the dates on which the algo predicted a closing, I would need a list with the same order. This means you have to think about when you get rid of columns because you might want them later on. 

And if you want to get the contents of a column in a nice comma seperated list, just add `.tolist()` to the end of it like so:

In [ ]:
percentile_list[percentile_list["pred"]==1]["accum"].tolist()